In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import tensorflow as tf
from keras.saving import register_keras_serializable

@register_keras_serializable()
def custom_loss_function(y_true, y_pred):
    """
    Custom loss function to ignore the last class (dummy) during loss calculation.
    """
    # Exclude the last class (dummy) from the loss calculation
    y_true = tf.cast(y_true[:, :-1], tf.float32)  # Exclude last class
    y_pred = y_pred[:, :-1]  # Exclude last class
    loss = tf.keras.backend.categorical_crossentropy(y_true, y_pred)  # Use tf.keras.backend
    return loss


In [3]:
import os
import csv
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import pandas as pd

# Preprocessing function: Resize images to (384, 384) and normalize pixel values to [-1, 1]
def preprocess_image(image_path):
    img = image.load_img(image_path, target_size=(384, 384))  # Resize to model's input size
    img_array = image.img_to_array(img)                       # Convert to array
    img_array = img_array / 255.0 
    img_array = img_array * 2 - 1  # Normalize pixel values to range [-1, 1]
    return img_array

# Function to load multiple models for each category
def load_models(model_paths):
    models = []
    for model_path in model_paths:
        try:
            model = load_model(model_path)
            models.append(model)
            print(f"Model loaded from: {model_path}")
        except Exception as e:
            print(f"Error loading model from {model_path}: {e}")
    return models

# Function to average predictions from multiple models
def average_predictions(models, images):
    all_predictions = []
    for model in models:
        predictions = model.predict(images)  # Get predictions from the model
        predictions = [preddd[:, :-1] for preddd in predictions]
        all_predictions.append(predictions)  # Remove the dummy class for this model's predictions
        
    
    # Averaging softmax probabilities across all models
    avg_predictions = [np.mean(np.array([pred[i] for pred in all_predictions]), axis=0) for i in range(len(all_predictions[0]))]
    return avg_predictions

# Function to predict attributes for images in batches and save to CSV
def predict_attributes_and_save_to_csv(image_folder, models, category, config, batch_size=128):
    num_attributes = config["num_attributes"]
    attribute_mapping = config["attribute_mapping"]
    csv_filename = f"{category.lower().replace(' ', '_')}.csv"
    image_files = sorted(
        [f for f in os.listdir(image_folder) if f.endswith('.jpg')],
        key=lambda x: int(x.split('.')[0])
    )  # Sort by numerical part of the file name

    # CSV header
    header = ['id', 'Category', 'len'] + [f'attr_{i+1}' for i in range(10)]

    # Open CSV file
    with open(csv_filename, mode='w', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=header)
        writer.writeheader()

        for i in range(0, len(image_files), batch_size):
            batch_files = image_files[i:i + batch_size]
            batch_images = np.array([preprocess_image(os.path.join(image_folder, file)) for file in batch_files])

            # Average predictions from all models
            avg_predictions = average_predictions(models, batch_images)

            # Extract predicted classes for each attribute
            predicted_classes = [np.argmax(pred, axis=-1) for pred in avg_predictions]

            for idx, file in enumerate(batch_files):
                # Extract image id by removing leading zeros and file extension
                image_id = str(int(file.split('.')[0])) if file != '000000.jpg' else '0'

                # Map predicted classes to attribute labels up to num_attributes
                predicted_labels = {
                    f'attr_{j + 1}': attribute_mapping[f'attr_{j + 1}'][predicted_classes[j][idx]]
                    for j in range(num_attributes)
                }

                # Prepare a row with dynamic attributes and dummy values for the rest
                row = {
                    'id': image_id,
                    'Category': category,
                    'len': num_attributes
                }
                for j in range(1, 11):
                    attr_key = f'attr_{j}'
                    row[attr_key] = predicted_labels.get(attr_key, 'dummy')

                # Write the row to the CSV
                writer.writerow(row)

# Generalized function to process all categories with ensemble models
def process_all_categories_with_ensembles(categories_config, model_paths_dict, base_image_folder, batch_size=128):
    for category, config in categories_config.items():
        print(f"Processing category: {category}")
        
        # Load all models for the category
        model_paths = model_paths_dict[category]
        models = load_models(model_paths)
        
        # Ensure at least one model is loaded
        if not models:
            print(f"No valid models found for category {category}. Skipping...")
            continue

        # Image folder for the category
        image_folder = os.path.join(base_image_folder, category.replace(" ", "_"))
        
        # Predict attributes and save to CSV
        predict_attributes_and_save_to_csv(image_folder, models, category, config, batch_size=batch_size)

# Combine all CSVs into a single submission file
def combine_csvs(categories, output_filename="submission.csv"):
    # Collect all CSV filenames generated
    csv_files = [f"{category.lower().replace(' ', '_')}.csv" for category in categories]

    # Read all CSV files into a single DataFrame
    df_list = [pd.read_csv(file) for file in csv_files]

    # Concatenate all DataFrames into one
    combined_df = pd.concat(df_list, ignore_index=True)

    # Convert 'id' column to numeric for proper sorting
    combined_df['id'] = pd.to_numeric(combined_df['id'], errors='coerce')

    # Sort the DataFrame by 'id' column in ascending order
    sorted_df = combined_df.sort_values(by='id')

    # Write the sorted DataFrame to a new CSV file
    sorted_df.to_csv(output_filename, index=False)

    print(f"Merged and sorted CSV file '{output_filename}' created successfully!")

# Example Usage
if __name__ == "__main__":
    # Define configurations for all categories
    categories_config = {
        "Men Tshirts": {
            "num_attributes": 5,
            "attribute_mapping": {
                'attr_1': {0: 'black', 1: 'default', 2: 'multicolor', 3: 'white', 4: 'dummy'}, 'attr_2': {0: 'polo', 1: 'round', 2: 'dummy'}, 'attr_3': {0: 'printed', 1: 'solid', 2: 'dummy'}, 'attr_4': {0: 'default', 1: 'solid', 2: 'typography', 3: 'dummy'}, 'attr_5': {0: 'long sleeves', 1: 'short sleeves', 2: 'dummy'}
                # Add other attribute mappings...
            },
        },
        "Sarees": {
            "num_attributes": 10,
            "attribute_mapping": {
                'attr_1': {0: 'default', 1: 'same as border', 2: 'same as saree', 3: 'solid', 4: 'dummy'}, 'attr_2': {0: 'default', 1: 'no border', 2: 'solid', 3: 'temple border', 4: 'woven design', 5: 'zari', 6: 'dummy'}, 'attr_3': {0: 'big border', 1: 'no border', 2: 'small border', 3: 'dummy'}, 'attr_4': {0: 'cream', 1: 'default', 2: 'green', 3: 'multicolor', 4: 'navy blue', 5: 'pink', 6: 'white', 7: 'yellow', 8: 'dummy'}, 'attr_5': {0: 'daily', 1: 'party', 2: 'traditional', 3: 'wedding', 4: 'dummy'}, 'attr_6': {0: 'default', 1: 'jacquard', 2: 'tassels and latkans', 3: 'dummy'}, 'attr_7': {0: 'default', 1: 'same as saree', 2: 'woven design', 3: 'zari woven', 4: 'dummy'}, 'attr_8': {0: 'default', 1: 'printed', 2: 'solid', 3: 'woven design', 4: 'zari woven', 5: 'dummy'}, 'attr_9': {0: 'applique', 1: 'botanical', 2: 'checked', 3: 'default', 4: 'elephant', 5: 'ethnic motif', 6: 'floral', 7: 'peacock', 8: 'solid', 9: 'dummy'}, 'attr_10': {0: 'no', 1: 'yes', 2: 'dummy'}                # Add other attribute mappings...
            },
        },
        "Kurtis": {
            "num_attributes": 9,
            "attribute_mapping": {
                'attr_1': {0: 'black', 1: 'blue', 2: 'green', 3: 'grey', 4: 'maroon', 5: 'multicolor', 6: 'navy blue', 7: 'orange', 8: 'pink', 9: 'purple', 10: 'red', 11: 'white', 12: 'yellow', 13: 'dummy'}, 'attr_2': {0: 'a-line', 1: 'straight', 2: 'dummy'}, 'attr_3': {0: 'calf length', 1: 'knee length', 2: 'dummy'}, 'attr_4': {0: 'daily', 1: 'party', 2: 'dummy'}, 'attr_5': {0: 'default', 1: 'net', 2: 'dummy'}, 'attr_6': {0: 'default', 1: 'solid', 2: 'dummy'}, 'attr_7': {0: 'default', 1: 'solid', 2: 'dummy'}, 'attr_8': {0: 'short sleeves', 1: 'sleeveless', 2: 'three-quarter sleeves', 3: 'dummy'}, 'attr_9': {0: 'regular', 1: 'sleeveless', 2: 'dummy'}                # Add other attribute mappings...
            },
        },
        "Women Tshirts": {
            "num_attributes": 8,
            "attribute_mapping": {
                'attr_1': {0: 'black', 1: 'default', 2: 'maroon', 3: 'multicolor', 4: 'pink', 5: 'white', 6: 'yellow', 7: 'dummy'}, 'attr_2': {0: 'boxy', 1: 'loose', 2: 'regular', 3: 'dummy'}, 'attr_3': {0: 'crop', 1: 'long', 2: 'regular', 3: 'dummy'}, 'attr_4': {0: 'default', 1: 'printed', 2: 'solid', 3: 'dummy'}, 'attr_5': {0: 'default', 1: 'funky print', 2: 'graphic', 3: 'quirky', 4: 'solid', 5: 'typography', 6: 'dummy'}, 'attr_6': {0: 'default', 1: 'long sleeves', 2: 'short sleeves', 3: 'dummy'}, 'attr_7': {0: 'cuffed sleeves', 1: 'regular sleeves', 2: 'dummy'}, 'attr_8': {0: 'applique', 1: 'default', 2: 'dummy'}                # Add other attribute mappings...
            },
        },
        "Women Tops & Tunics": {
            "num_attributes": 10,
            "attribute_mapping": {
                'attr_1': {0: 'black', 1: 'blue', 2: 'default', 3: 'green', 4: 'maroon', 5: 'multicolor', 6: 'navy blue', 7: 'peach', 8: 'pink', 9: 'red', 10: 'white', 11: 'yellow', 12: 'dummy'}, 'attr_2': {0: 'boxy', 1: 'default', 2: 'fitted', 3: 'regular', 4: 'dummy'}, 'attr_3': {0: 'crop', 1: 'regular', 2: 'dummy'}, 'attr_4': {0: 'default', 1: 'high', 2: 'round neck', 3: 'square neck', 4: 'stylised', 5: 'sweetheart neck', 6: 'v-neck', 7: 'dummy'}, 'attr_5': {0: 'casual', 1: 'party', 2: 'dummy'}, 'attr_6': {0: 'default', 1: 'printed', 2: 'solid', 3: 'dummy'}, 'attr_7': {0: 'default', 1: 'floral', 2: 'graphic', 3: 'quirky', 4: 'solid', 5: 'typography', 6: 'dummy'}, 'attr_8': {0: 'long sleeves', 1: 'short sleeves', 2: 'sleeveless', 3: 'three-quarter sleeves', 4: 'dummy'}, 'attr_9': {0: 'default', 1: 'puff sleeves', 2: 'regular sleeves', 3: 'sleeveless', 4: 'dummy'}, 'attr_10': {0: 'applique', 1: 'default', 2: 'knitted', 3: 'ruffles', 4: 'tie-ups', 5: 'waist tie-ups', 6: 'dummy'}                # Add other attribute mappings...
            },
        }
    }

    # Define model paths for each category
    model_paths_dict = {
        "Men Tshirts": [
            "/kaggle/input/model-v1-m/model_checkpoint.keras",
            "/kaggle/input/model-v2-m/model_checkpoint.keras",
            "/kaggle/input/model-v3-m/model_checkpoint.keras",
            "/kaggle/input/model-v4-m/model_checkpoint.keras"
                
        ],
        "Sarees": [
            "/kaggle/input/model-v1-s/model_checkpoint.keras",
            "/kaggle/input/model-v2-s/model_checkpoint.keras",
            "/kaggle/input/model-v3-s/model_checkpoint.keras",
            "/kaggle/input/model-v4-s/model_checkpoint.keras"
        ],
        "Kurtis": [
            "/kaggle/input/model-v1-k/model_checkpoint.keras",
            "/kaggle/input/model-v2-k/model_checkpoint.keras",
            "/kaggle/input/model-v3-k/model_checkpoint.keras",
            "/kaggle/input/model-v4-k/model_checkpoint.keras"
        ],
        "Women Tshirts": [
            "/kaggle/input/model-v1-w/model_checkpoint.keras",
            "/kaggle/input/model-v2-w/model_checkpoint.keras",
            "/kaggle/input/model-v3-w/model_checkpoint.keras",
            "/kaggle/input/model-v4-w/model_checkpoint.keras"
        ],
        "Women Tops & Tunics": [
            "/kaggle/input/model-v1-wt/model_checkpoint.keras",
            "/kaggle/input/model-v2-wt/model_checkpoint.keras",
            "/kaggle/input/model-v3-wt/model_checkpoint.keras",
            "/kaggle/input/model-v4-wt/model_checkpoint.keras"
        ]
    }

    # Define the base image folder
    base_image_folder = "/kaggle/input/sorted-test/sorted_images"

    # Process all categories
    process_all_categories_with_ensembles(categories_config, model_paths_dict, base_image_folder)

    # Combine all CSVs into a single submission file
    combine_csvs(categories_config.keys(), output_filename="submission.csv")


Processing category: Men Tshirts
Model loaded from: /kaggle/input/model-v1-m/model_checkpoint.keras
Model loaded from: /kaggle/input/model-v2-m/model_checkpoint.keras
Model loaded from: /kaggle/input/model-v3-m/model_checkpoint.keras


/opt/conda/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'adamw', because it has 1812 variables whereas the saved optimizer has 1816 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model loaded from: /kaggle/input/model-v4-m/model_checkpoint.keras


I0000 00:00:1731772548.678521     297 service.cc:145] XLA service 0x7eb8a4017ff0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1731772548.678566     297 service.cc:153]   StreamExecutor device (0): NVIDIA L4, Compute Capability 8.9
I0000 00:00:1731772548.678571     297 service.cc:153]   StreamExecutor device (1): NVIDIA L4, Compute Capability 8.9
I0000 00:00:1731772548.678573     297 service.cc:153]   StreamExecutor device (2): NVIDIA L4, Compute Capability 8.9
I0000 00:00:1731772548.678576     297 service.cc:153]   StreamExecutor device (3): NVIDIA L4, Compute Capability 8.9
I0000 00:00:1731772552.406196     398 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_3', 20 bytes spill stores, 20 bytes spill loads

I0000 00:00:1731772552.507813     406 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_3', 228 bytes spill stores, 22

1/4 ━━━━━━━━━━━━━━━━━━━━ 1:09 23s/step

...

3/3 ━━━━━━━━━━━━━━━━━━━━ 13s 7s/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 12s 6s/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 12s 6s/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 21s 10s/step
Processing category: Women Tops & Tunics
Model loaded from: /kaggle/input/model-v1-wt/model_checkpoint.keras
Model loaded from: /kaggle/input/model-v2-wt/model_checkpoint.keras
Model loaded from: /kaggle/input/model-v3-wt/model_checkpoint.keras
Model loaded from: /kaggle/input/model-v4-wt/model_checkpoint.keras


I0000 00:00:1731775985.482284   38907 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_8', 20 bytes spill stores, 20 bytes spill loads

I0000 00:00:1731775985.569537   38909 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_8', 260 bytes spill stores, 260 bytes spill loads



4/4 ━━━━━━━━━━━━━━━━━━━━ 13s 160ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 12s 161ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 12s 161ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 22s 123ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 171ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 172ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 172ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 145ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 172ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 172ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 172ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 146ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 172ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 172ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 172ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 146ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 172ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 172ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 172ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 146ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 172ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 172ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 172ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 145ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 172ms/step
4/4 ━━━━━━━━━━━━━━━━━

I0000 00:00:1731776320.049902   49923 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_8', 4 bytes spill stores, 4 bytes spill loads



1/1 ━━━━━━━━━━━━━━━━━━━━ 18s 18s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 12s 12s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 12s 12s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 24s 24s/step
Merged and sorted CSV file 'submission.csv' created successfully!


In [4]:
print("done")

done
